<a href="https://colab.research.google.com/github/DimpleB0501/ERA1/blob/main/S2/ERA_V1_S2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch # import pytorch 
import torch.nn as nn  # import all the functions that are used in neural networks models. https://pytorch.org/tutorials/beginner/nn_tutorial.html
import torch.nn.functional as F #https://pytorch.org/docs/stable/nn.functional.html
import torch.optim as optim # Implemented optimization functions. https://pytorch.org/docs/stable/optim.html
from torchvision import datasets, transforms # The torchvision package consists of popular datasets, model architectures, and common image transformations for computer vision.
!pip install torchsummary
from torchsummary import summary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
use_cuda = torch.cuda.is_available() # check if the system is on cpu or gpu
device = torch.device("cuda" if use_cuda else "cpu") # select the device as gpu if available 
device

device(type='cuda')

In [3]:
batch_size = 128 # datasize is 128 while processing it in batches

# load the training and apply transforms like normalization and converting it to a tensor
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,)) # use mean of and std deivation of training set
                    ])),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 98926664.04it/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 44129578.81it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 24931295.09it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 4437579.49it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



# Some Notes on our naive model

We are going to write a network based on what we have learnt so far. 

The size of the input image is 28x28x1. We are going to add as many layers as required to reach RF = 32 "atleast". 

In [4]:
class FirstDNN(nn.Module):
  def __init__(self):
    super(FirstDNN, self).__init__()
    # r_in:1, n_in:28, j_in:1, s:1, r_out:3, n_out:28, j_out:1
    self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
    # r_in:3 , n_in:28 , j_in:1 , s:1 , r_out:5 , n_out:28 , j_out:1
    self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
    # r_in:5 , n_in:28 , j_in:1 , s:2 , r_out:6 , n_out:14 , j_out:2
    self.pool1 = nn.MaxPool2d(2, 2)
    # r_in:6 , n_in:14 , j_in:2 , s:1 , r_out:10 , n_out:14 , j_out:2
    self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
    # r_in:10 , n_in:14 , j_in:2 , s:1 , r_out:14 , n_out:14 , j_out:2
    self.conv4 = nn.Conv2d(128, 256, 3, padding = 1)
    # r_in:14 , n_in:14 , j_in:2 , s:2 , r_out:16 , n_out:7 , j_out:4
    self.pool2 = nn.MaxPool2d(2, 2)
    # r_in:16 , n_in:7 , j_in:4 , s:1 , r_out:24 , n_out:5 , j_out:4
    self.conv5 = nn.Conv2d(256, 512, 3)
    # r_in:24 , n_in:5 , j_in:4 , s:1 , r_out:32 , n_out:3 , j_out:4
    self.conv6 = nn.Conv2d(512, 1024, 3)
    # r_in:32 , n_in:3 , j_in:4 , s:1 , r_out:40 , n_out:1 , j_out:4
    self.conv7 = nn.Conv2d(1024, 10, 3)

  def forward(self, x):
    x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
    x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
    x = F.relu(self.conv6(F.relu(self.conv5(x))))
    #x = F.relu(self.conv7(x))
    x = self.conv7(x)
    x = x.view(-1, 10)
    return F.log_softmax(x)


In [5]:
model = FirstDNN().to(device)

In [6]:
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

<ipython-input-4-2feb7a2ccac7>:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [7]:
!pip install torchscan

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 52.4 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.1+cu118 requires torch==2.0.0, but you have torch 1.13.1 which is incompatible.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 1.

In [8]:
from torchscan import summary
summary(model,(1, 28, 28), receptive_field=True, max_depth=1)     

__________________________________________________________________________
Layer       Type         Output Shape         Param #      Receptive field
firstdnn    FirstDNN     (-1, 10)             0            1              
├─conv1     Conv2d       (-1, 32, 28, 28)     320          3              
├─conv2     Conv2d       (-1, 64, 28, 28)     18,496       5              
├─pool1     MaxPool2d    (-1, 64, 14, 14)     0            6              
├─conv3     Conv2d       (-1, 128, 14, 14)    73,856       10             
├─conv4     Conv2d       (-1, 256, 14, 14)    295,168      14             
├─pool2     MaxPool2d    (-1, 256, 7, 7)      0            16             
├─conv5     Conv2d       (-1, 512, 5, 5)      1,180,160    24             
├─conv6     Conv2d       (-1, 1024, 3, 3)     4,719,616    32             
├─conv7     Conv2d       (-1, 10, 1, 1)       92,170       40             
Trainable params: 6,379,786
Non-trainable params: 0
Total params: 6,379,786
------------------------

<ipython-input-4-2feb7a2ccac7>:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [9]:
from tqdm import tqdm # training models with progress bars
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)  # progress depends on data and batch size
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device) # upload data and labels to gpu (or cpu)
        optimizer.zero_grad()  # https://pytorch.org/docs/stable/generated/torch.optim.Optimizer.zero_grad.html, optimizer.zero_grad() clears x.grad for every parameter x in the optimizer. It’s important to call this before loss.backward(), otherwise you’ll accumulate the gradients from multiple passes.
        output = model(data)  # process the data through the model
        loss = F.nll_loss(output, target) # calculate the loss
        loss.backward() #  computes dloss/dx for every parameter x which has requires_grad=True. These are accumulated into x.grad for every parameter x. In pseudo-code:x.grad += dloss/dx
        optimizer.step() # perform parameter update, optimizer.step updates the value of x using the gradient x.grad. For example, the SGD optimizer performs:x += -lr * x.grad
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device) # upload data and labels to gpu (or cpu)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [10]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)  # using sgd optimizer https://pytorch.org/docs/stable/generated/torch.optim.SGD.html

for epoch in range(1, 2): # number of times the training needs to be performed
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-4-2feb7a2ccac7>:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.03726774826645851 batch_id=468: 100%|██████████| 469/469 [00:35<00:00, 13.15it/s]



Test set: Average loss: 0.0525, Accuracy: 9835/10000 (98%)

